# COMP0087 group project

This is the main .ipynb file for the project "Coherence Improvement on Pre-Training Based Cooking Recipe Generation", by group 17053487. 

## Before start
- It is better for you to first take a glimpse at our report since we managed to explain our code by  referring to some chapters in our report.

We use this main file to organize our codes and avoid uploading too many notebook files. This file is designed to help you understand generally:

1. How our code was structured.

2. How our experiment was conducted. The procedures include,
  - Environment Setup. 
  - Data Preprocessing. 
  - Model setup
  - Model training. 
  - Model evaluation. 

This file is just an example of our experiment, which means the final results are not complete. Specifically, this file does not contain, 

1. Some of the helper functions. Since our helper functions are designed with respect to various model structures, this notebook only contains helper functions for BART1+GRU model. 
  - Help functions for data preprocessing is in *data_preprocess.py*
  - Batch processing functions are saved in *batch_process.py*. This file mainly contains our processing during training procedure. 

2. Codes for all the models. In our report, we proposed a novel model. But to test how different modules in this model influence the overall performance, we conducted experiments based on various model structures. We have an example code for BART with GRU in this notebook, but you can find all other models' codes either by looking into the independent *.py* files or previous experiment notebooks in our Github. The python files are oraganized as follows,
  - The codes for differnt models structures can be found in *model_setup.py*. 
  - The codes of utility functions for various models (e.g. model initialization, model saving) can be found in *model_utils.py*
  - The training codes for different models are in *model_train.py*

3. All the experiment notebooks. To find those, you can look into our zipped file *experiments_nbs.zip* to find them. 

4. The fine-tuned model files. The models are complex thus contatin many parameters. So we decided not to upload it on Moodle. We upload all our trained model to huggingface (https://huggingface.co/jky594176). We also create a separate evaluation file for your reference. Please refer to the *.ipynb* file to evaluate our model. 

Also, we uploaded everything to a github () in case something is missing. You can find the all necessary files there. 


## Environment Setup

In [ ]:
!pip install transformers
!pip install boto3
!git clone https://github.com/NVIDIA/apex
!pip install jiwer==2.2.0
!pip install -U nltk
%cd apex
!pip install -v --no-cache-dir ./
%cd .. 


     |████████████████████████████████| 133kB 8.1MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.26.5
    Uninstalling urllib3-1.26.5:
      Successfully uninstalled urllib3-1.26.5
fatal: destination path 'apex' already exists and is not an empty directory.
Requirement already up-to-date: nltk in /usr/local/lib/python3.7/dist-packages (3.6.2)
/content/apex
Created temporary directory: /tmp/pip-ephem-wheel-cache-kseyb97h
Created temporary directory: /tmp/pip-req-tracker-d_qobi6k
Created requirements tracker '/tmp/pip-req-tracker-d_qobi6k'
Created temporary directory: /tmp/pip-install-3iawuriw
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-pd3pjtgn
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-d_qobi6k'
    Running setup.py (path:/tmp/pip-req-build-pd3pjtgn/setup.py) egg_info for package from file:///content/apex
    Running command

In [ ]:
# avoid GPU crushing
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

# mount the google drive to colab
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
# data preprocessing
from sklearn.model_selection import train_test_split
import pandas as pd
import re
import numpy as np
import json
import h5py

# Output Control
import argparse
import logging
from tqdm import trange

# pytorch
import torch
print(torch.__version__)
import torch.nn.functional as F

# transformers
from transformers import GPT2Config
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import BartForCausalLM, BartTokenizer,BartConfig

1.8.1+cu101


## data preprocessing

In [ ]:
# the original dataframe
df_ori = pd.read_csv("./gdrive/MyDrive/COMP0087/full_dataset.csv")

In [ ]:
# Sample a subset of the whole data for preprocessing. 
# We do not wish to process the whole dataset due to the limitation of computation resources

np.random.seed(0)
train_size = 350000
test_size = 100
sample_train = 500000
sample_test = int(test_size*2)

subset_idx = np.random.choice(range(len(df_ori)),sample_train+sample_test,replace=False)
train_idx = subset_idx[0:sample_train]
test_idx = subset_idx[sample_train:]
df = df_ori.loc[subset_idx ,:]
# df_test = df_ori.loc[subset_idx[-100:],:]

In [ ]:
def data_drop(df):
  """
  Data clearning:
  remove samples those are not suitable for training, as mentioned in chapter 4.1
  :param df:input dataframe
  :return:dataframe after removal
  """

  # calculate length of the removal items
  # the conditions are applied to title, ingredients and directions
  remove1 = df.loc[df.title.map(lambda x: len(x)<4 )]
  remove2 = df.loc[df.ingredients.map(lambda x: len(x)<2)]
  remove3 = df.loc[df.directions.map(lambda x: len(x) < 2 or len(''.join(x)) < 30)]
  remove4 = df.loc[df.directions.map(lambda x: re.search('(step|mix all)', ''.join(str(x)), re.IGNORECASE)!=None)]
  len(remove3)+len(remove2)+len(remove1)+len(remove4)
  # remove the items
  df.drop(df[df.title.map(lambda x: len(x)<4)].index, inplace=True)
  df.drop(df[df.ingredients.map(lambda x: len(x)<2)].index, inplace=True)
  df.drop(df[df.directions.map(lambda x: len(x) < 2 or len(''.join(x)) < 30)].index, inplace=True)
  df.drop(df[df.directions.map(lambda x: re.search('(step|mix all)', ''.join(str(x)), re.IGNORECASE)!=None)].index, inplace=True)

  return df

In [ ]:
# Clean invalid recipes in the dataset, as mentioned in chapter 4.1 of the report.
df = data_drop(df)
# train-test split, we will not use full testing set.
df.reset_index(inplace=True)
train, test_sp = train_test_split(df, test_size=0.05, random_state = 7) 

# Testing samples saved for final evaluation, 100 samples seem like a small size
# but we will discuss why we only use 100 samples.in chapter 5.2
test = test_sp[0:100]

In [ ]:
def df_to_plaintext_file(input_df, output_file, train = True):
    """
    prepare data in txt type for further use
    :param input_df: input dataframe
    :param output_file: output file name
    :param train: train or test sample processing
    :return: None
    """
    print("Writing to", output_file)
    
    with open(output_file, 'w') as f:
        for index, row in input_df.iterrows():
            if index%100000==0:
                print(index)
                print(res)
            if type(row.NER)!=str:
                continue
            title = row.title
            directions = json.loads(row.directions)

            # we drop the directions with a length less than 2
            if len(directions) <= 1 and train:
              continue
            # print(len(directions))
            ingredients = json.loads(row.ingredients)
            ner = json.loads(row.NER)
            # print(ner)
            # contol tokens added
            res = " <RECIPE_START> <INPUT_START> " + " <NEXT_INPUT> ".join(ner) + " <INPUT_END> <INGR_START> " + \
              " <NEXT_INGR> ".join(ingredients) + " <INGR_END> <INSTR_START> " + \
              " <NEXT_INSTR> ".join(directions) + " <INSTR_END> <TITLE_START> " + title + " <TITLE_END> <RECIPE_END>"
            # print(res)
            f.write("{}\n".format(res))


def init_tokenizer(tokenizer_type = "facebook/bart-base"):
  
  tokenizer = BartTokenizer.from_pretrained(tokenizer_type)
  special_tokens = {
    "additional_special_tokens": [
        "<TITLE_START>",
        "<TITLE_END>",
        "<INSTR_START>",
        "<NEXT_INSTR>",
        "<INSTR_END>",
        "<INGR_START>",
        "<NEXT_INGR>",
        "<INGR_END>",
        "<RECIPE_START>",
        "<RECIPE_END>",
        "<INPUT_START>",
        "<INPUT_END>",
        "<NEXT_INPUT>"
      ]
  }
  tokenizer.add_special_tokens(special_tokens)
  return tokenizer



def datapre_txt2h5_CLM(tokenizer,train_size, test_size):
    """
    prepare data from txt to h5 file for casual language modelling
    :param tokenizer: pretrained tokenizer type
    :param train_size: training size
    :param test_size: testing size
    :return: None
    """

    # special tokens identified
    end_token_id = tokenizer.convert_tokens_to_ids(["<RECIPE_END>"])[0]
    ing_token_id = tokenizer.convert_tokens_to_ids(["<INPUT_END>"])[0]
    # set the maximum directions size
    directions_size = 512
    hf = h5py.File("unsupervised.h5", "w")
    for filename in ["test", "train"]:
        out_np = []
        data = open("unsupervised_" + filename + ".txt", "r")
        num = 0
        rows = 0
        for line in data:
            num += 1
            if num % 10000 == 0:
                print("Processed " + str(num) + " Valid: " + str(rows))
            text_tokens = tokenizer(line)['input_ids']

            # this if statement is important for Conditional Generation
            # or it will leads to a gradient overflow problem
            if ing_token_id in text_tokens:
                pass
            else:
                continue

            # error in one recipe
            # 50273 is the token for <RECIPE_END>
            if len(text_tokens) > directions_size or (50273 not in text_tokens):
                continue
            text_tokens_ids = text_tokens

            # padding the token with <RECIPE_END> to match the direction size
            while len(text_tokens_ids) < directions_size:
                text_tokens_ids.append(end_token_id)
            out_np.append(text_tokens_ids)
            rows += 1

            # when enough valid samples was processed, break the loop
            if rows == train_size and filename == 'train':
                print("training sample enough:", train_size)
                break
            if rows == test_size and filename == 'test':
                print("testing sample enough", test_size)
                break
        out_mat = np.matrix(out_np)
        print(out_mat.shape)
        hf.create_dataset(filename, data=out_mat)
    hf.close()


In [ ]:
# Save data after preprocessing

# we saved the dataframe files to txt file for further use
df_to_plaintext_file(train, 'unsupervised_train.txt')
df_to_plaintext_file(test, 'unsupervised_test.txt',train = False)

# tokenizer initialization and convert txt file to h5 file. 
# The tokenization happens simultaneously with the conversion.
tokenizer = init_tokenizer("facebook/bart-base")
datapre_txt2h5_CLM(tokenizer,train_size, test_size)

Writing to unsupervised_train.txt
100000
 <RECIPE_START> <INPUT_START> garlic <NEXT_INPUT> Spanish onion <NEXT_INPUT> stalks celery <NEXT_INPUT> ground chuck <NEXT_INPUT> tomato puree <NEXT_INPUT> tomato paste <NEXT_INPUT> parmesan cheese <NEXT_INPUT> romano cheese <NEXT_INPUT> cabernet sauvignon <NEXT_INPUT> course ground black pepper <NEXT_INPUT> bay leaves <NEXT_INPUT> thyme <NEXT_INPUT> oregano <NEXT_INPUT> salt <NEXT_INPUT> sugar <NEXT_INPUT> pasta <INPUT_END> <INGR_START> 6 cloves garlic, finely chopped <NEXT_INGR> 1 medium Spanish onion, finely diced <NEXT_INGR> 3 stalks celery, finely diced <NEXT_INGR> 3 lbs (1.4 kg). ground chuck (I use round steak for lower fat) <NEXT_INGR> 4 15 oz (420 grm). cans tomato puree <NEXT_INGR> 1 6 oz (168 grm). can tomato paste <NEXT_INGR> freshly grated parmesan cheese <NEXT_INGR> freshly grated romano cheese <NEXT_INGR> 1/3 bottle cabernet sauvignon (or burgundy or bordeaux) <NEXT_INGR> 1-1/2 tsp (7 ml) course ground black pepper <NEXT_INGR> 2 b


(97, 512)


Token indices sequence length is longer than the specified maximum sequence length for this model (1161 > 1024). Running this sequence through the model will result in indexing errors


Processed 10000 Valid: 9370
Processed 20000 Valid: 18759
Processed 30000 Valid: 28103
Processed 40000 Valid: 37415
Processed 50000 Valid: 46833
Processed 60000 Valid: 56206
Processed 70000 Valid: 65504
Processed 80000 Valid: 74890
Processed 90000 Valid: 84269
Processed 100000 Valid: 93643
Processed 110000 Valid: 103044
Processed 120000 Valid: 112421
Processed 130000 Valid: 121789
Processed 140000 Valid: 131161
Processed 150000 Valid: 140560
Processed 160000 Valid: 149939
Processed 170000 Valid: 159314
Processed 180000 Valid: 168756
Processed 190000 Valid: 178113
Processed 200000 Valid: 187447
Processed 210000 Valid: 196815
Processed 220000 Valid: 206196
Processed 230000 Valid: 215666
Processed 240000 Valid: 225029
Processed 250000 Valid: 234410
Processed 260000 Valid: 243827
Processed 270000 Valid: 253244
Processed 280000 Valid: 262596
Processed 290000 Valid: 271958
Processed 300000 Valid: 281367
Processed 310000 Valid: 290760
Processed 320000 Valid: 300177
Processed 330000 Valid: 3095

# fine-tune with BART

In [ ]:
# fine-tuning model with bart
# logging info
import logging
logger = logging.getLogger(__name__)

import glob
import logging
import random
import gc
import boto3
import shutil

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler
from torch.utils.data.distributed import DistributedSampler
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm, trange
import logging
import math
import os
from dataclasses import dataclass, field
from typing import Optional
from torch.utils.data import Dataset
import h5py
import torch

from transformers import WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup
from transformers import BartForCausalLM, BartTokenizer,BartConfig,BartForConditionalGeneration

In [ ]:
# data loading
class TextDataset(Dataset):
    # prepare data in the torch type
    def __init__(self, tokenizer, file_path='train', block_size=512):
        cached_features_file = "unsupervised.h5"

        logger.info("Loading features from cached file %s", cached_features_file)
        with h5py.File(cached_features_file, 'r') as f:
            if file_path=='test':
                self.examples = f[file_path][:] #this is a dev set, 10% of a test set
            else:
                self.examples = f[file_path][:]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item])

def load_and_cache_examples(args, tokenizer, evaluate=False):
    dataset = TextDataset(tokenizer, file_path="test" if evaluate else "train", block_size=args.block_size)
    print(dataset[0:5])
    return dataset

In [ ]:
# parameters setting, modified from the example of huggingface transformer

parser = argparse.ArgumentParser()
## Required parameters
parser.add_argument("--train_data_file", default="unsupervised.h5", type=str, required=False,
                        help="The input training data file (a text file).")
parser.add_argument("--output_dir", default="./gdrive/MyDrive/COMP0087/Bart1Gru_CP_0529", type=str, required=False,
                        help="The output directory where the model predictions and checkpoints will be written.")

## Other parameters
parser.add_argument("--model_type", default="facebook/bart-base" ,type=str,   #"facebook/bart-base"
                        help="The model architecture to be fine-tuned.")
parser.add_argument("--model_name_or_path", default="facebook/bart-base", type=str,
                        help="The model checkpoint for weights initialization.")

parser.add_argument("--eval_data_file", default=None, type=str,
                        help="An optional input evaluation data file to evaluate the perplexity on (a text file).")

parser.add_argument("--config_name", default="", type=str,
                        help="Optional pretrained config name or path if not the same as model_name_or_path")
parser.add_argument("--tokenizer_name", default="", type=str,
                        help="Optional pretrained tokenizer name or path if not the same as model_name_or_path")
parser.add_argument("--cache_dir", default="", type=str,
                        help="Optional directory to store the pre-trained models downloaded from s3 (instread of the default one)")
parser.add_argument("--block_size", default=-1, type=int,
                        help="Optional input sequence length after tokenization."
                             "The training dataset will be truncated in block of this size for training."
                             "Default to the model max input length for single sentence inputs (take into account special tokens).")
parser.add_argument("--do_train", action='store_true',
                        help="Whether to run training.")
parser.add_argument("--do_eval", action='store_true',
                        help="Whether to run eval on the dev set.")
parser.add_argument("--evaluate_during_training", action='store_true',
                        help="Run evaluation during training at each logging step.")
parser.add_argument("--do_lower_case", action='store_true',
                        help="Set this flag if you are using an uncased model.")

parser.add_argument("--per_gpu_train_batch_size", default=8, type=int,
                        help="Batch size per GPU/CPU for training.")
parser.add_argument("--per_gpu_eval_batch_size", default=4, type=int,
                        help="Batch size per GPU/CPU for evaluation.")
parser.add_argument('--gradient_accumulation_steps', type=int, default=1,
                        help="Number of updates steps to accumulate before performing a backward/update pass.")
parser.add_argument("--learning_rate", default=2e-4, type=float,
                        help="The initial learning rate for Adam.")
parser.add_argument("--weight_decay", default=0.0, type=float,
                        help="Weight deay if we apply some.")
parser.add_argument("--adam_epsilon", default=1e-7, type=float,
                        help="Epsilon for Adam optimizer.")
parser.add_argument("--max_grad_norm", default=1.0, type=float,
                        help="Max gradient norm.")
parser.add_argument("--num_train_epochs", default=2, type=float,
                        help="Total number of training epochs to perform.")
parser.add_argument("--max_steps", default=-1, type=int,
                        help="If > 0: set total number of training steps to perform. Override num_train_epochs.")
parser.add_argument("--warmup_steps", default=0, type=int,
                        help="Linear warmup over warmup_steps.")

parser.add_argument('--logging_steps', type=int, default=50,
                        help="Log every X updates steps.")
parser.add_argument('--save_steps', type=int, default=50,
                        help="Save checkpoint every X updates steps.")
parser.add_argument("--eval_all_checkpoints", action='store_true',
                        help="Evaluate all checkpoints starting with the same prefix as model_name_or_path ending and ending with step number")
parser.add_argument("--no_cuda", action='store_true',
                        help="Avoid using CUDA when available")
parser.add_argument('--overwrite_output_dir', action='store_true',
                        help="Overwrite the content of the output directory")
parser.add_argument('--overwrite_cache', action='store_true',
                        help="Overwrite the cached training and evaluation sets")
parser.add_argument("--aws_bucket", default="", type=str,
                        help="Whether to upload to specified bucket.")
args, unknown = parser.parse_known_args()
  
  

In [ ]:
import random

def shuffle_instruction(this_batch,ins_element_id,recipe_end_id):
  """
  shuffle function to produce incoherent samples
  :param this_batch: The batch to be processed
  :param ins_element_id: The token of <NEXT_INSTR>
  :param recipe_end_id: The token of <RECIPE_END>
  :return: shuffled batch and feedback signal, as discussed in chapter 3.3 of report
  """

  # find the end of recipe
  end_index = (this_batch == recipe_end_id).nonzero()[0]

  # find the position of <NEXT_INSTR> tokens.
  ing_index = (this_batch == ins_element_id).nonzero()
  ing_index = ing_index[ing_index<end_index]

  # If more than 2 steps of directions found, shuffle this sample.
  if len(ing_index) > 1:
    # convert sentence to list, then
    # use random.shuffle function to shuffle the list
    split_result = torch.tensor_split(this_batch, ing_index.squeeze())
    split_list = list(split_result[1:-1])
    random.shuffle(split_list)
    shuffle_batch = torch.cat((split_result[0],torch.cat(split_list),split_result[-1]))
    return shuffle_batch, 1
  else: 
    return this_batch, 0


def ins_token_idf(this_batch,ins_element_id,recipe_end_id):
  """
  identify the <NEXT_INSTR> token in a batch, this is necessary for CDM module
  :param this_batch: The batch to be processed
  :param ins_element_id: The token of <NEXT_INSTR>
  :param recipe_end_id: The token of <RECIPE_END>
  :return: the identified position of <NEXT_INSTR>
  """
  if (this_batch == recipe_end_id).nonzero().size()[0] == 0:
    return None
  end_index = (this_batch == recipe_end_id).nonzero()[0]
  ing_index = (this_batch == ins_element_id).nonzero()
  ing_index = ing_index[ing_index<end_index]
  if len(ing_index) > 0:
    return ing_index
  else: 
    return None



def split_ing_dirs(this_batch):
  """
  Only used in Conditional generation; for Casual LM it is not.
  Split ingredients and directions, to respectively prepare
  the input of encoder and decoder sides. (for conditional generation)
  :param this_batch: the input batch
  :return: the processed ingredients and directions
  """
  token_list = tokenizer.convert_tokens_to_ids(["<INSTR_START>",
                                                "<INGR_START>",
                                                "<NEXT_INSTR>",
                                                "<RECIPE_END>",
                                                "<INPUT_END>",
                                                "<RECIPE_START>"])
  ins_start_id = token_list[0]
  ing_start_id = token_list[1]
  ins_element_id = token_list[2]
  recipe_end_id = token_list[3]
  input_end_id = token_list[4]
  recipe_start_id = token_list[5]

  sample = this_batch.clone()

  # ingredients size and instructions (directions) size
  ing_size = 48
  ins_size = 512

  # identify the ingredients and instructions in a batch
  ins_index = (sample == recipe_start_id).nonzero()[0] - 1

  # construct a tensor to store the ingredients
  ingredients = torch.zeros(ing_size)
  ingredients = torch.full_like(ingredients, input_end_id)
  if ins_index > ing_size:
    print("size overflow")
    ingredients[0:ing_size] = this_batch[0:ing_size]
  else:
    ingredients[0:ins_index] = this_batch[0:ins_index]
  sample[0:ins_index - 1] = 0
  nz = sample.nonzero().squeeze()
  # move the non-zero elements of instructions vector to left
  directions = torch.zeros(sample.numel() - nz.numel())
  directions = torch.full_like(directions, recipe_end_id)
  # directions = this_batch[-1]
  directions = torch.cat((sample[nz], directions))
  directions[0] = 0
  del sample
  return ingredients, directions



## Model setup

Here we give an example of our model. This is "BART1+GRU" model in our report. 

In [ ]:
import torch
import torch.nn as nn
# from transformer import AlbertTokenizer, AlbertForMultipleChoice
from transformers import BartModel, BartTokenizer, BartConfig, BartForCausalLM
from torch.nn import CrossEntropyLoss, MSELoss, GRU, BCELoss
from torch.nn.utils.rnn import pad_sequence


class bartWithGRU(nn.Module):
    """
    Args:
    tokenizer: the tokenizer for backbone model
    token_size: the size of a batch
    """
    def __init__(self, tokenizer, token_size=512):
        super(bartWithGRU, self).__init__()

        # device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
        self.bart = BartForCausalLM.from_pretrained("facebook/bart-base")
        self.bart.resize_token_embeddings(len(tokenizer))

        # The module for consistency checking
        # embedding layer. Though finally found not helpful
        # self.emb_size = 64
        # self.sentence_emb = nn.Linear(self.bart.config.d_model, self.emb_size)

        # The gru to capture discriminative feature
        self.gru = nn.GRU(self.bart.config.d_model, 32 , 1 ,batch_first = False)
        self.gru_head = nn.Linear(32, 1)
        self.linear_activation = nn.Sigmoid()
        self.cuda()

    def forward(self, batch, inst_pos, labels=None):
        """
        Args: 
        batch: the input batch
        inst_pos: the position of token <NEXT_INSTR> in every batch, as a index list.
        """

        # extract the last hidden state of decoder BART
        outputs = self.bart(batch,output_hidden_states=True)
        last_hidden = outputs["hidden_states"][-1]
        tokens_list = []
        # get the hidden state for the token <NEXT_INSTR>
        for i in range(batch.size()[0]):
          batch_hidden = last_hidden[inst_pos[i][:,0],inst_pos[i][:,1],:]
          tokens_list.append(batch_hidden)
        
        # padding for gru input
        gru_input = pad_sequence(tokens_list)
        # gru_input = self.sentence_emb(gru_input)
        gru_output,_ = self.gru(gru_input)
        
        # binary classfication for the coherence
        consist_output = self.gru_head(gru_output[-1])
        consist_output = self.linear_activation(consist_output)

        # output of next-token prediction
        logits = outputs["logits"]

        return logits, consist_output

In [ ]:
def save_model(args,model,tokenizer,model_class,tokenizer_class):
  """
  function taken from hugging-face transformer
  """
  # Create output directory if neede
  if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

  logger.info("Saving model checkpoint to %s", args.output_dir)
  # Save a trained model, configuration and tokenizer using `save_pretrained()`.
  # They can then be reloaded using `from_pretrained()`
  model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
  model_to_save.save_pretrained(args.output_dir)
  tokenizer.save_pretrained(args.output_dir)

  # Good practice: save your training arguments together with the trained model
  torch.save(args, os.path.join(args.output_dir, 'training_args.bin'))

  # Load a trained model and vocabulary that you have fine-tuned
  model = model_class.from_pretrained(args.output_dir)
  tokenizer = tokenizer_class.from_pretrained(args.output_dir, do_lower_case=args.do_lower_case)
  # model.to(args.device)


In [ ]:
def model_init(args,logger, model_class, tokenizer):
  """
  mode initialization function modified from HuggingFace transformer
  """
  if args.eval_data_file is None and args.do_eval:
    raise ValueError("Cannot do evaluation without an evaluation data file. Either supply a file to --eval_data_file "
                         "or remove the --do_eval argument.")
  if os.path.exists(args.output_dir) and os.listdir(args.output_dir) and args.do_train and not args.overwrite_output_dir:
    raise ValueError("Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(args.output_dir))
  device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
  args.n_gpu = torch.cuda.device_count()
  args.device = device

  # Setup logging
  logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                        datefmt = '%m/%d/%Y %H:%M:%S',
                        level = logging.INFO)

  # customized model initialization
  model = bartWithGRU(tokenizer)

  if args.block_size <= 0:
    args.block_size = tokenizer.max_len_single_sentence  # Our input block size will be the max possible for the model
  args.block_size = min(args.block_size, tokenizer.max_len_single_sentence)
  model.to(args.device)

  logger.info("Training/evaluation parameters %s", args)
  return model, logger

## Model training

In [ ]:
from torch.nn import CrossEntropyLoss, MSELoss
def train(args, train_dataset, model, tokenizer):
    """ Train the model """
    """
    This train function is specially designed for training Casual Language Modeling. 
    The basic stucutre is taken from the HuggingFace transformer, but the details are 
    largely modified to fit our purpose, especially when processing each batch. 
    Please look through this function mainly focusing on the batch processing. 

    :param args: the parameters
    :param train_dataset: training dataset
    :param model: Model for training
    :param tokenizer: tokenizer model
    :return: loss, global step and a example batch. 
    """

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
         'weight_decay': args.weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps,
                                                num_training_steps=t_total)

    try:
        from apex import amp
    except ImportError:
        raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")

    model, optimizer = amp.initialize(model, optimizer, opt_level="O2")

    # Train!
    global_step = 0
    epoch = 0
    tr_loss, logging_loss = 0.0, 0.0

    # hyparameters for modified loss, check this in our chapter 3.4 of the report
    alpha = 1.2
    lmbda = 0.7
    model.zero_grad()

    train_iterator = trange(int(args.num_train_epochs), desc="Epoch", disable=True)
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=True)
        
        # in the second round of training, the CDM is fixed, and the lambda increases
        # to heavily penalize the incoherent sentence.
        if epoch == 1:
          lmbda = 1.5
          fix_layers = [model.gru, model.gru_head, model.linear_activation]
                # mybart.units_logit, mybart.units_activation]
          for layer in fix_layers:
            for parameter in layer.parameters():
              parameter.requires_grad = False


        epoch +=1
        for step, batch in enumerate(epoch_iterator):

            batch_drop = 0
            token_list = tokenizer.convert_tokens_to_ids(["<INSTR_START>",
                                                          "<INGR_START>",
                                                          "<NEXT_INSTR>",
                                                          "<RECIPE_END>"])
            ins_start_id = token_list[0]
            ing_start_id = token_list[1]
            ins_element_id = token_list[2]
            recipe_end_id = token_list[3]


            # randomly shuffle the sentence
            random_shuffle = torch.randint(2, (batch.size()[0], 1),device = args.device)

            tokens_list = []
            max_length = 0

            # process the batch for training
            for batch_no in range(len(batch)):
              # <INSTR_START> not found in the batch, this is an invalid sample
              if (batch[batch_no] == ins_start_id).nonzero().size()[0] == 0:
                print("An error happens, break")
                batch_drop = 1
                break
              # shuffle the batch
              if random_shuffle[batch_no] == 1:
                shuffle_batch, shuffle_result = shuffle_instruction(batch[batch_no],
                                                      ins_start_id,ing_start_id)
                random_shuffle[batch_no] = shuffle_result
              else:
                shuffle_batch = batch[batch_no]
              
              # find the <NEXT_INSTR> in the batch
              ins_pos = ins_token_idf(batch[batch_no],
                                      ins_element_id, recipe_end_id)
              # if <NEXT_INSTR> not found, invalid
              if ins_pos == None:
                batch_drop = 1
                break

              ins_pos = ins_pos.reshape(-1,1)

              # add the <NEXT_INSTR> to tokens_list for model input
              if ins_pos.size()[0] > max_length:
                max_length = ins_pos.size()[0]
              ins_idx = torch.hstack((torch.full_like(ins_pos,batch_no),ins_pos))
              tokens_list.append(ins_idx)
              
            if batch_drop == 1:
              print("An error in this batch, break")
              torch.cuda.empty_cache()
              continue

            # for the purpose of text generation
            # the input and the label should be shifted by 1 token.
            inputs, labels = (batch[:, 0:-1], batch[:, 1:])
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)

            model.train()

            loss_CE = CrossEntropyLoss(reduction = "sum")
            loss_BCE = BCELoss()

            # model feed-forward
            logits, consist_output = model(inputs,tokens_list)



            # loss adjustment
            bart_vocab_size = 50278
            loss1 = 0
            loss2 = 0

            for i in range(len(batch)):
              ing_end_index = (inputs[i] == ins_start_id).nonzero()[0]

              # modify the weights of different parts of the generation, as mentioned in chapter 3.4 
              loss1 += alpha * loss_CE(logits[i,:ing_end_index].reshape(-1, bart_vocab_size), 
                               labels[i,:ing_end_index].reshape(-1))
              
              loss1 += loss_CE(logits[i,ing_end_index:].reshape(-1, bart_vocab_size), 
                               labels[i,ing_end_index:].reshape(-1))
              
            loss1 = loss1/(len(batch)*511)
            loss2 = loss_BCE(consist_output, random_shuffle.float())
            # control the weights of 2 training tasks.
            loss = loss1 + lmbda * loss2

            # output every 100 steps. This is designed to avoid browser crush
            if global_step % 100 == 0:
              logger.info(step)
              print("step", global_step)
              print("loss:",loss.item())

            # save every 10000 step.
            if global_step % 10000 == 0 and global_step !=0:
              model_checkpoint = model.bart
              model_class = BartForCausalLM
              tokenizer_class = BartTokenizer
              save_model(args,model_checkpoint,tokenizer,model_class,tokenizer_class)


            # automatic loss scale
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()

            tr_loss += loss.item()
            if (step) % args.gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

            if args.max_steps > 0 and global_step > 2:
                epoch_iterator.close()
                break
            del inputs, labels,logits,consist_output , loss1,loss2, loss
            torch.cuda.empty_cache()

    return global_step, tr_loss / global_step, batch

In [ ]:
torch.manual_seed(0)
# training dataset initialization, model initialization 
train_dataset = load_and_cache_examples(args, tokenizer, evaluate=False)
mybart, logger = model_init(args,logger,BartForCausalLM,tokenizer)
# And train!
global_step, tr_loss, batch = train(args, train_dataset, mybart, tokenizer)

tensor([[    0, 50273, 50275,  ..., 50274, 50274, 50274],
        [    0, 50273, 50275,  ..., 50274, 50274, 50274],
        [    0, 50273, 50275,  ..., 50274, 50274, 50274],
        [    0, 50273, 50275,  ..., 50274, 50274, 50274],
        [    0, 50273, 50275,  ..., 50274, 50274, 50274]])


05/30/2021 13:09:02 - INFO - filelock -   Lock 140069682789584 acquired on /root/.cache/huggingface/transformers/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.3c4e938c08a46506ff9a4be12e0f858cd714b9a5c76090d571e7a4aa95cae853.lock


05/30/2021 13:09:02 - INFO - filelock -   Lock 140069682789584 released on /root/.cache/huggingface/transformers/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.3c4e938c08a46506ff9a4be12e0f858cd714b9a5c76090d571e7a4aa95cae853.lock


05/30/2021 13:09:03 - INFO - filelock -   Lock 140074209625488 acquired on /root/.cache/huggingface/transformers/486355ec722ef05fd480e999d4c763be56549ae930f6a3742ee721a5d2a05647.9faea28a6782a9589c09b1942c039943df02232d83d2ac288a69ddfa928eae22.lock


05/30/2021 13:09:13 - INFO - filelock -   Lock 140074209625488 released on /root/.cache/huggingface/transformers/486355ec722ef05fd480e999d4c763be56549ae930f6a3742ee721a5d2a05647.9faea28a6782a9589c09b1942c039943df02232d83d2ac288a69ddfa928eae22.lock


...

step 86100
loss: 0.4071299135684967
An error in this batch, break
An error in this batch, break


05/31/2021 02:00:01 - INFO - __main__ -   43602


step 86200
loss: 0.5667463541030884


05/31/2021 02:00:54 - INFO - __main__ -   43702


step 86300
loss: 0.7890177965164185


In [ ]:
# save the model
args.output_dir = "./gdrive/MyDrive/COMP0087/Bart1Gru_CP_0529"
model = mybart.bart
model_class = BartForCausalLM
tokenizer_class = BartTokenizer
save_model(args,model,tokenizer,model_class,tokenizer_class)

05/31/2021 02:01:26 - INFO - __main__ -   Saving model checkpoint to ./gdrive/MyDrive/COMP0087/Bart1Gru_CP_0529


## model Evaluation

In [ ]:
# if evaluation is independent of the training, load the file
# model = BartForCausalLM.from_pretrained(args.output_dir)

## A generation example

This generation example is only suitable for BART1 typed model. For BART2 typed model, other generation helper functions should be used.

In [ ]:
#model loading
#tokenizer = AutoTokenizer.from_pretrained("mbien/recipenlg")
#model = AutoModelWithLMHead.from_pretrained("mbien/recipenlg")

In [ ]:
raw_text = 'tomato,egg'
prepared_input = ' <RECIPE_START> <INPUT_START> ' + raw_text.replace(',', ' <NEXT_INPUT> ').replace(';', ' <INPUT_END>')
tokenizer.encode(prepared_input)[0:-1]

[0, 50273, 50275, 33063, 3938, 50277, 38299]

In [ ]:
def set_seed(args):
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k > 0: keep only top k tokens with highest probability (top-k filtering).
            top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
        From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
    """
    assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits


def sample_sequence(model, length, context, tokenizer, num_samples=1, temperature=1, top_k=0, top_p=0.0, device='cpu'):
    end_token = tokenizer.convert_tokens_to_ids(["<RECIPE_END>"])[0]
    context = torch.tensor(context, dtype=torch.long, device=device)
    context = context.unsqueeze(0).repeat(num_samples, 1)
    generated = context
    with torch.no_grad():
        for _ in range(length):
            inputs = {'input_ids': generated}
            outputs = model(**inputs)  # Note: we could also use 'past' with GPT-2/Transfo-XL/XLNet (cached hidden-states)
            # print(outputs)
            next_token_logits = outputs["logits"][0, -1, :] / temperature
            
            filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
            next_token = torch.multinomial(F.softmax(filtered_logits, dim=-1), num_samples=1)
            generated = torch.cat((generated, next_token.unsqueeze(0)), dim=1)
            #print(next_token)
            if next_token.item() == end_token:
                break
    return generated

In [ ]:
# raw_text = args.prompt if args.prompt else input("Comma-separated ingredients, semicolon to close the list >>> ")
model.to("cuda")
raw_text = 'potato;'

prepared_input = ' <RECIPE_START> <INPUT_START> ' + raw_text.replace(',', ' <NEXT_INPUT> ').replace(';', ' <INPUT_END>')
context_tokens = tokenizer.encode(prepared_input)[0:-1]
out = sample_sequence(
            model=model,
            context=context_tokens,
            tokenizer=tokenizer,
            length=512,
            device = "cuda"
        )
out = out[0, len(context_tokens):].tolist()
text = tokenizer.decode(out, clean_up_tokenization_spaces=True)
if "<RECIPE_END>" not in text:
  print(text)
  print("Failed to generate, recipe's too long")
  full_text = prepared_input + text
  print(full_text)
else:
  full_text = prepared_input + text
  print(full_text)
  markdown = re.sub("<RECIPE_(START|END)>", "", full_text)
  recipe_n_title = markdown.split("<TITLE_START>")
  title = "# " + recipe_n_title[1].replace("<TITLE_END>", "") + " #\n"
  markdown = recipe_n_title[0].replace("<INPUT_START>", "## Input ingredients ##\n`").replace("<INPUT_END>", "`\n")
  markdown = markdown.replace("<NEXT_INPUT>", "`\n`").replace("<INGR_START>", "## Ingredients ##\n* ").replace("<NEXT_INGR>", "\n* ").replace("<INGR_END>", "\n")
  markdown = markdown.replace("<INSTR_START>", "## Instructions ##\n1) ").replace("<NEXT_INSTR>", "\n1) ").replace("<INSTR_END>", "\n")
  markdown = re.sub("$ +#", "#", markdown)
  markdown = re.sub("( +`|` +)", "`", markdown)
  print(title+markdown)#


 <RECIPE_START> <INPUT_START> potato <INPUT_END><INGR_START> 1 potato peeled, fork-tender and cut into small chunks <INGR_END> <INSTR_START> Add ingredients in above order in as listed. <NEXT_INSTR> Bake in mcher case oven at 425 for 12 Finally. <INSTR_END> <TITLE_START> Potato Potatoes - Quick!! <TITLE_END> <RECIPE_END>
#  Potato Potatoes - Quick!!   #
  ## Input ingredients ##
`potato`
## Ingredients ##
*  1 potato peeled, fork-tender and cut into small chunks 
 ## Instructions ##
1)  Add ingredients in above order in as listed. 
1)  Bake in mcher case oven at 425 for 12 Finally. 
 


In [ ]:
# gold standard
#df_eval = gold_st
model.to(args.device)
test_input = test.reset_index()
df_eval_ner = test_input["NER"]  #NER
df_eval_dir = test_input["directions"]

In [ ]:
def get_raw_input(NER):
    test_str = NER
    test_str = test_str.replace("[","")
    test_str = test_str.replace("]","")
    test_str = test_str.replace("\"","")

    return test_str

def get_instr(markdown):
  markdown = markdown.split("\n")
  if ' ## Instructions ##' in markdown:
    instr_index = markdown.index(' ## Instructions ##')
    
    output = [i for i in markdown[instr_index+1:]]

    if " " in output:
        output.remove(" ")

    if "" in output:
        output.remove("")    
    
    return output

  elif '## Instructions ##' in markdown:
    instr_index = markdown.index('## Instructions ##')

    output = [i for i in markdown[instr_index+1:]]

    if " " in output:
        output.remove(" ")
        
    if "" in output:
        output.remove("")    

    return output
    
  else:
    return ["failed to generate"]


def generate_recipe(raw_text):

    prepared_input = ' <RECIPE_START> <INPUT_START> ' + raw_text.replace(',', ' <NEXT_INPUT> ').replace(';', ' <INPUT_END>')
    context_tokens = tokenizer.encode(prepared_input)[0:-1]
    out = sample_sequence(
                model=model,
                context=context_tokens,
                tokenizer=tokenizer,
                length=800,
                device = args.device
            )
    out = out[0, len(context_tokens):].tolist()
    text = tokenizer.decode(out, clean_up_tokenization_spaces=True)
    if "<RECIPE_END>" not in text:
      # print(text)
      print("Failed to generate, recipe's too long")
      full_text = prepared_input + text
      # print(full_text)
      return generate_recipe(raw_text)
    else:
      full_text = prepared_input + text
      # print(full_text)
      markdown = re.sub("<RECIPE_(START|END)>", "", full_text)
      recipe_n_title = markdown.split("<TITLE_START>")
      if len(recipe_n_title)<=1:
        return generate_recipe(raw_text)
      title = "# " + recipe_n_title[1].replace("<TITLE_END>", "") + " #\n"
      markdown = recipe_n_title[0].replace("<INPUT_START>", "## Input ingredients ##\n`").replace("<INPUT_END>", "`\n")
      markdown = markdown.replace("<NEXT_INPUT>", "`\n`").replace("<INGR_START>", "## Ingredients ##\n* ").replace("<NEXT_INGR>", "\n* ").replace("<INGR_END>", "\n")
      #markdown = markdown.replace("<INSTR_START>", "## Instructions ##\n\"").replace("<NEXT_INSTR>", "\n\"").replace("<INSTR_END>", "\n")
      markdown = markdown.replace("<INSTR_START>", "## Instructions ##\n").replace("<NEXT_INSTR>", "\n").replace("<INSTR_END>", "\n")
      markdown = re.sub("$ +#", "#", markdown)
      markdown = re.sub("( +`|` +)", "`", markdown)
  
      return markdown

In [ ]:
df_eval_dir

0     ["Chop the tomatoes and cucumber into chunks."...
1     ["Preheat the over to 325 degrees F.", "Slice ...
2     ["Cook ground beef, onion, green pepper and ga...
3     ["Preheat an 8-inch-square nonstick waffle iro...
4     ["Soften shallots in large frying pan over med...
                            ...                        
95    ["Set a large pot or Dutch oven over high heat...
96    ["Dirt:", "Grind Oreos in food processor or ro...
97    ["Combine eggnog and Kahlua in 1 1/2-quart pit...
98    ["Bake the cake mix as directions say in an 11...
99    ["Cut tomatoes in 1/2-inch slices. Place slice...
Name: directions, Length: 100, dtype: object

In [ ]:
torch.manual_seed(1)
# generated 100*10 recipes from our test set
test_size = 100
replicated_size = 10
directions =  [[] for i in range(test_size)]

for i in range(test_size):
  raw_text = get_raw_input(df_eval_ner[i])
  print("\n Generating the recipe: ",i)

  for j in range(replicated_size):
    print("recipe: ",j)
    md = generate_recipe(raw_text)
    # get_instr convert the generated tokens to the markdown file
    directions[i].append(get_instr(md))


 Generating the recipe:  0
recipe:  0
recipe:  1
recipe:  2
recipe:  3
recipe:  4
recipe:  5
recipe:  6
recipe:  7
recipe:  8
recipe:  9

 Generating the recipe:  1
recipe:  0
recipe:  1
recipe:  2
recipe:  3
recipe:  4
recipe:  5
recipe:  6
recipe:  7
recipe:  8
recipe:  9

 Generating the recipe:  2
recipe:  0
recipe:  1
recipe:  2
recipe:  3
recipe:  4
recipe:  5
recipe:  6
recipe:  7
recipe:  8
recipe:  9

 Generating the recipe:  3
recipe:  0
recipe:  1
recipe:  2
recipe:  3
recipe:  4
recipe:  5
recipe:  6
recipe:  7
recipe:  8
recipe:  9

 Generating the recipe:  4
recipe:  0
recipe:  1
recipe:  2
recipe:  3
recipe:  4
recipe:  5
recipe:  6
recipe:  7
recipe:  8
recipe:  9

 Generating the recipe:  5
recipe:  0
recipe:  1
recipe:  2
recipe:  3
recipe:  4
recipe:  5
recipe:  6
recipe:  7
recipe:  8
recipe:  9

 Generating the recipe:  6
recipe:  0
recipe:  1
recipe:  2
recipe:  3
recipe:  4
recipe:  5
recipe:  6
recipe:  7
recipe:  8
recipe:  9

 Generating the recipe:  7
recipe

In [ ]:
#func of cos_sim
import re
import math
from collections import Counter


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    word = re.compile(r'\w+')
    words = word.findall(text)
    return Counter(words)


def get_result(content_a, content_b):
    text1 = content_a
    text2 = content_b

    vector1 = text_to_vector(text1)
    vector2 = text_to_vector(text2)

    cosine_result = get_cosine(vector1, vector2)
    return cosine_result

In [ ]:
#cosine_similarity

def COS_SIM(df_eval_dir,directions):
  """
  df_eval_dir := gold_standard recipes
  directions  := corresponding recipes """

  avg = 0

  for i in range(len(directions)):
    best = 0
    for j in range(len(directions[i])):
      cos = get_result(" ".join(eval(df_eval_dir[i])),
                      " ".join(directions[i][j]))
      best = max(best, cos)

    avg += best

  avg = avg/len(directions)

  print("avg:", avg)

In [ ]:
COS_SIM(df_eval_dir,directions)

avg: 0.5555631646129268


In [ ]:
import nltk
import nltk.translate.bleu_score as bleu
from nltk.translate.bleu_score import SmoothingFunction

import nltk.translate.gleu_score as gleu
import nltk.translate.meteor_score as meteor

In [ ]:
# Helper Func.
def list_to_words(recipe):
  words = []
  for i in recipe:
    words += i.split()

  return words

# New BLEU/GLEU
def bleu_score(recipe, refer):
    hyp = list_to_words(eval(recipe))
    refs = []
    for i in refer:
        refs.append(list_to_words(i))

    smoothie = SmoothingFunction().method5
    score_ref_a = bleu.sentence_bleu(refs, hyp, smoothing_function=smoothie, weights=(1,0,0,0))
    return score_ref_a

def gleu_score(recipe, refer):
    hyp = list_to_words(eval(recipe))
    refs = []
    for i in refer:
        refs.append(list_to_words(i))

    score_ref_a = gleu.sentence_gleu(refs, hyp)
    return score_ref_a

In [ ]:
bleu_avg = []
gleu_avg = []

for i in range(len(directions)):
  # print(bleu_score(df_eval_dir[i], directions[i]))
  bleu_avg.append(bleu_score(df_eval_dir[i], directions[i]))

for i in range(len(directions)):
  gleu_avg.append(gleu_score(df_eval_dir[i], directions[i]))
print(np.mean(bleu_avg))
print(np.mean(gleu_avg))

0.8079534431023818
0.10138513219581695


## Upload to Huggingface

In [ ]:
!sudo apt-get install git-lfs
!pip install huggingface_hub
!transformers-cli login
!git config --global user.email "uclqjia@ucl.ac.uk"
!git config --global user.name "jky594176"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (1,659 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-p

In [ ]:
model_type = ["recipe_GPT2","recipe_BART1","recipe_BART2","recipe_BART1_NN","recipe_BART1_GRU"]
output_dir = "./gdrive/MyDrive/COMP0087/Bart1Gru_CP_350k"
model = BartForCausalLM.from_pretrained(output_dir)
model.push_to_hub(model_type[4])

05/31/2021 05:50:07 - INFO - huggingface_hub.repository -   git version 2.17.1
Sorry, no usage text found for "git-lfs"
05/31/2021 05:50:25 - INFO - huggingface_hub.repository -   
Git LFS: (0 of 1 files) 0 B / 366.70 MB                                        
Git LFS: (0 of 1 files) 43.45 KB / 366.70 MB                                   
Git LFS: (0 of 1 files) 327.45 KB / 366.70 MB                                  
Git LFS: (0 of 1 files) 3.04 MB / 366.70 MB                                    
Git LFS: (0 of 1 files) 8.81 MB / 366.70 MB                                    
Git LFS: (0 of 1 files) 17.85 MB / 366.70 MB                                   
Git LFS: (0 of 1 files) 25.18 MB / 366.70 MB                                   
Git LFS: (0 of 1 files) 34.28 MB / 366.70 MB                                   
Git LFS: (0 of 1 files) 41.89 MB / 366.70 MB                                   
Git LFS: (0 of 1 files) 49.92 MB / 366.70 MB                                   
Git LFS: (0 of 1 fi

## Blessing

In [ ]:
# Divine beast bless no bug here! 
#         ┌─┐    ┌─┐
#      ┌─┘ ┴───┘ ┴──┐
#      │                   │
#      │       ───       │
#      │  ─┬┘     └┬─  │
#      │                   │
#      │       ─┴─       │
#      │                   │
#      └─┐         ┌───┘
#          │         │
#          │         │
#          │         │
#          │         └──────────────┐
#          │                                  │
#          │                                  ├─┐
#          │                                  ┌─┘
#          │                                  │
#          └─┐  ┐  ┌──────┬──┐  ┌──┘
#            │  ─┤ ─┤         │  ─┤ ─┤
#            └──┴──┘         └──┴──┘